In [1]:
import ray
import cProfile
import pstats
import time
import sys
import numpy as np

In [2]:
ray.init()

Waiting for redis server at 127.0.0.1:50991 to respond...
Waiting for redis server at 127.0.0.1:58744 to respond...
Starting local scheduler with the following resources: {'CPU': 8, 'GPU': 0}.

View the web UI at http://localhost:8890/notebooks/ray_ui98543.ipynb?token=90637ba3347332c21b694ea350c9de92c66ccb89e39dac63



{'local_scheduler_socket_names': ['/tmp/scheduler42608744'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store55972419', manager_name='/tmp/plasma_manager58394320', manager_port=24235)],
 'redis_address': '127.0.0.1:50991',
 'webui_url': 'http://localhost:8890/notebooks/ray_ui98543.ipynb?token=90637ba3347332c21b694ea350c9de92c66ccb89e39dac63'}

# Functions

In [3]:
def get_profile_data(arr):
    pr = cProfile.Profile(timeunit = .000001)
    pr.enable()
    ray.put(arr)
    pr.disable()
    return pr

In [4]:
def stats_printer(profile_object):
    s = StringIO.StringIO()
    sortby = 'module'
    ps = pstats.Stats(profile_object, stream=s).sort_stats(sortby)
    ps.strip_dirs()
    ps.print_stats('worker|plasma')
    print(s.getvalue())

In [5]:
def print_profile_data(arr):
    profile_data = get_profile_data(arr)
    
    print('ncalls', 'tottime', 'inlinetime', 'filename(function)', sep='\t\t')
    for line in profile_data.getstats():
        code = line.code
        if type(code) is not str:
            code = code.co_filename.split('/')[-1]+":"+code.co_name

        ncalls = line.callcount
        tottime = "%.6f" % line.totaltime
        inlinetime = "%.6f" % line.inlinetime

        print(ncalls, tottime, inlinetime, code, sep='\t\t')

# Test Inputs

In [6]:
short_numpy_array = np.array([1,2,3,4,5])
large_numpy_array = np.array([i for i in range(100000)])
very_large_numpy_array = np.array([i for i in range(1000000)])

In [7]:
sys.getsizeof(short_numpy_array)

136

In [8]:
sys.getsizeof(large_numpy_array)

800096

In [9]:
sys.getsizeof(very_large_numpy_array)

8000096

# Profiling Results with Custom Printer

In [14]:
print_profile_data(short_numpy_array)

ncalls		tottime		inlinetime		filename(function)
2		0.000002		0.000002		<method 'items' of 'dict' objects>
4		0.000001		0.000001		<built-in method builtins.isinstance>
1		0.000000		0.000000		<built-in method _thread.get_ident>
1		0.000001		0.000001		threading.py:name
1		0.000002		0.000001		threading.py:getName
1		0.000003		0.000003		threading.py:current_thread
1		0.000000		0.000000		<method 'disable' of '_lsprof.Profiler' objects>
1		0.000564		0.000012		worker.py:store_and_register
1		0.000015		0.000015		<method 'compute_put_id' of 'local_scheduler.LocalSchedulerClient' objects>
1		0.000000		0.000000		<method 'id' of 'common.ObjectID' objects>
1		0.000569		0.000005		worker.py:put_object
1		0.000008		0.000003		worker.py:check_main_thread
1		0.000001		0.000001		worker.py:check_connected
1		0.000001		0.000001		worker.py:__init__
1		0.000007		0.000001		worker.py:__enter__
1		0.000008		0.000003		worker.py:__exit__
1		0.000003		0.000002		worker.py:log_span
2		0.000001		0.000001		worker.py:<di

In [11]:
print_profile_data(large_numpy_array)

ncalls		tottime		inlinetime		filename(function)
2		0.000001		0.000001		<method 'items' of 'dict' objects>
4		0.000004		0.000004		<built-in method builtins.isinstance>
1		0.000001		0.000001		<built-in method _thread.get_ident>
1		0.000001		0.000001		threading.py:name
1		0.000002		0.000001		threading.py:getName
1		0.000003		0.000002		threading.py:current_thread
1		0.000000		0.000000		<method 'disable' of '_lsprof.Profiler' objects>
1		0.011899		0.000013		worker.py:store_and_register
1		0.000017		0.000017		<method 'compute_put_id' of 'local_scheduler.LocalSchedulerClient' objects>
1		0.000001		0.000001		<method 'id' of 'common.ObjectID' objects>
1		0.011907		0.000006		worker.py:put_object
1		0.000010		0.000005		worker.py:check_main_thread
1		0.000001		0.000001		worker.py:check_connected
1		0.000002		0.000002		worker.py:__init__
1		0.000010		0.000002		worker.py:__enter__
1		0.000018		0.000006		worker.py:__exit__
1		0.000004		0.000002		worker.py:log_span
2		0.000002		0.000002		worker.py:<di

In [12]:
print_profile_data(very_large_numpy_array)

ncalls		tottime		inlinetime		filename(function)
2		0.000001		0.000001		<method 'items' of 'dict' objects>
4		0.000001		0.000001		<built-in method builtins.isinstance>
1		0.000001		0.000001		<built-in method _thread.get_ident>
1		0.000001		0.000001		threading.py:name
1		0.000002		0.000001		threading.py:getName
1		0.000003		0.000002		threading.py:current_thread
1		0.000000		0.000000		<method 'disable' of '_lsprof.Profiler' objects>
1		0.034082		0.000012		worker.py:store_and_register
1		0.000018		0.000018		<method 'compute_put_id' of 'local_scheduler.LocalSchedulerClient' objects>
1		0.000001		0.000001		<method 'id' of 'common.ObjectID' objects>
1		0.034089		0.000007		worker.py:put_object
1		0.000008		0.000003		worker.py:check_main_thread
1		0.000002		0.000002		worker.py:check_connected
1		0.000001		0.000001		worker.py:__init__
1		0.000008		0.000001		worker.py:__enter__
1		0.000014		0.000004		worker.py:__exit__
1		0.000003		0.000002		worker.py:log_span
2		0.000001		0.000001		worker.py:<di

# Profiling Results with Default Printer

In [15]:
get_profile_data(short_numpy_array).print_stats()

         27 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 threading.py:1076(name)
        1    0.000    0.000    0.000    0.000 threading.py:1150(getName)
        1    0.000    0.000    0.000    0.000 threading.py:1230(current_thread)
        1    0.000    0.000    0.000    0.000 worker.py:2128(__init__)
        1    0.000    0.000    0.000    0.000 worker.py:2134(__enter__)
        1    0.000    0.000    0.000    0.000 worker.py:2141(__exit__)
        1    0.000    0.000    0.000    0.000 worker.py:2155(log_span)
        2    0.000    0.000    0.000    0.000 worker.py:2163(log)
        2    0.000    0.000    0.000    0.000 worker.py:2184(<dictcomp>)
        1    0.000    0.000    0.001    0.001 worker.py:2241(put)
        1    0.000    0.000    0.001    0.001 worker.py:262(store_and_register)
        1    0.000    0.000    0.001    0.001 worker.py:327(

In [16]:
get_profile_data(large_numpy_array).print_stats()

         27 function calls in 0.003 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 threading.py:1076(name)
        1    0.000    0.000    0.000    0.000 threading.py:1150(getName)
        1    0.000    0.000    0.000    0.000 threading.py:1230(current_thread)
        1    0.000    0.000    0.000    0.000 worker.py:2128(__init__)
        1    0.000    0.000    0.000    0.000 worker.py:2134(__enter__)
        1    0.000    0.000    0.000    0.000 worker.py:2141(__exit__)
        1    0.000    0.000    0.000    0.000 worker.py:2155(log_span)
        2    0.000    0.000    0.000    0.000 worker.py:2163(log)
        2    0.000    0.000    0.000    0.000 worker.py:2184(<dictcomp>)
        1    0.000    0.000    0.003    0.003 worker.py:2241(put)
        1    0.000    0.000    0.003    0.003 worker.py:262(store_and_register)
        1    0.000    0.000    0.003    0.003 worker.py:327(

In [17]:
get_profile_data(very_large_numpy_array).print_stats()

         27 function calls in 0.006 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 threading.py:1076(name)
        1    0.000    0.000    0.000    0.000 threading.py:1150(getName)
        1    0.000    0.000    0.000    0.000 threading.py:1230(current_thread)
        1    0.000    0.000    0.000    0.000 worker.py:2128(__init__)
        1    0.000    0.000    0.000    0.000 worker.py:2134(__enter__)
        1    0.000    0.000    0.000    0.000 worker.py:2141(__exit__)
        1    0.000    0.000    0.000    0.000 worker.py:2155(log_span)
        2    0.000    0.000    0.000    0.000 worker.py:2163(log)
        2    0.000    0.000    0.000    0.000 worker.py:2184(<dictcomp>)
        1    0.000    0.000    0.006    0.006 worker.py:2241(put)
        1    0.000    0.000    0.006    0.006 worker.py:262(store_and_register)
        1    0.000    0.000    0.006    0.006 worker.py:327(